# Process TTE-related variables from the CESM-LE

In [1]:
%matplotlib inline
import os
import shutil

from glob import glob

import cftime

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

import intake
import pop_tools
import esmlab
import util

import warnings
warnings.filterwarnings('ignore')

## Spin up dask cluster

In [42]:
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=2, # The number of cores you want
    memory='256 GB', # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=2:mem=256GB', # Specify resources
    project='NCGD0011', # Input your project ID here
    walltime='02:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)
# Scale up
cluster.scale(32)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [44]:
client

Client Scheduler: tcp://10.12.206.28:39636 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/8787/status,Cluster Workers: 3 Cores: 6 Memory: 715.26 GiB


In [45]:
grid = pop_tools.get_grid('POP_gx1v6')
grid

<xarray.Dataset>
Dimensions:      (nlat: 384, nlon: 320, nreg: 14, z_t: 60, z_w: 60, z_w_bot: 60)
Coordinates:
  * z_t          (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
  * z_w          (z_w) float64 0.0 1e+03 2e+03 3e+03 ... 4.75e+05 5e+05 5.25e+05
  * z_w_bot      (z_w_bot) float64 1e+03 2e+03 3e+03 ... 5e+05 5.25e+05 5.5e+05
  * nreg         (nreg) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13
Dimensions without coordinates: nlat, nlon
Data variables: (12/15)
    TLAT         (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG        (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    ULAT         (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG        (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
    DXT          (nlat, nlon) float64 1.894e+06 1.893e+06 ... 1.473e+06
    DYT          (nlat, nlon) float64 5.94e+06 5.94e+06 ... 5.046e+06 5.046e+06
    ...           ...
    UAREA        (nlat, nlon) float64 1.423e+13 1.423e+13 ... 7.639e+12
    KMT          (nlat, nlon) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    REGION_MASK  (nlat, nlon) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    dz           (z_t) float64 1e+03 1e+03 1e+03 ... 2.499e+04 2.5e+04 2.5e+04
    region_name  (nreg) <U21 'Caspian Sea' 'Black Sea' ... 'Hudson Bay'
    region_val   (nreg) int64 -14 -13 -12 -5 1 2 3 4 6 7 8 9 10 11
Attributes:
    lateral_dims:       [384, 320]
    vertical_dims:      60
    vert_grid_file:     gx1v7_vert_grid
    horiz_grid_fname:   inputdata/ocn/pop/gx1v6/grid/horiz_grid_20010402.ieeer8
    topography_fname:   inputdata/ocn/pop/gx1v6/grid/topography_20090204.ieeei4
    region_mask_fname:  inputdata/ocn/pop/gx1v6/grid/region_mask_20090205.ieeei4
    type:               dipole
    title:              POP_gx1v6 grid

## Read the CESM-LE data 

We will use [`intake-esm`](https://intake-esm.readthedocs.io/en/latest/), which is a data catalog tool.
It enables querying a database for the files we want, then loading those directly as an `xarray.Dataset`.

First step is to set the "collection" for the CESM-LE, which depends on a json file conforming to the [ESM Catalog Specification](https://github.com/NCAR/esm-collection-spec).

In [46]:
#catalog_file = '/glade/work/mgrover/intake-esm-catalogs/cesm_le_bgc.json'
catalog_file = '/glade/u/home/kristenk/TTE_CESM-LE/krill-cesm-le/notebooks/data/glade-cesm1-le.json'
#catalog_file = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm1-le.json'
variables = ['DIA_IMPVF_Fe','HDIFB_Fe','WT_Fe','KPP_SRC_Fe'] #QSW_HTP','SHF_QSW','QSW_HBL'] #'diatC', 'spC', 'zooC'] #, 'TEMP','IFRAC', 
             #'graze_diat', 'graze_sp', 'graze_diaz']

experiments = ['20C', 'RCP85']
stream = 'pop.h.ecosys.nyear1' #'pop.h'
    
col = intake.open_esm_datastore(catalog_file, sep=',')
col

,unique
experiment,7
case,108
component,6
stream,15
variable,1052
date_range,116
member_id,40
path,191066
ctrl_branch_year,6
ctrl_experiment,4


Now we will search the collection for the ensemble members (unique `member_id`'s) that have a chlorophyll field. This is necessary because the ocean biogeochemistry was corrupted in some members and the data deleted.

In this cell, `member_id` is a list of the ensemble members we want to operate on.

In [47]:
col_sub = col.search(experiment=['20C'],                      
                     stream='pop.h', 
                     variable=['diatChl'])

member_id = list(col_sub.df.member_id.unique())
print(member_id)

[1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 101, 102, 103, 104, 105]


## Now let's search for the data we want

Specify a list of variables and perform a search. Under the hood, the `search` functionality uses [`pandas`](https://pandas.pydata.org/) data frames. We can view that frame here using the `.df` syntax.

In [48]:
col_sub = col.search(
    experiment=experiments, 
    stream=stream, 
    variable=variables,
    member_id=member_id,
    )

print(col_sub)

col_sub.df.head()

<glade-cesm1-le catalog with 2 dataset(s) from 380 asset(s)>


,experiment,case,component,stream,variable,date_range,member_id,path,ctrl_branch_year,ctrl_experiment,ctrl_member_id
0,20C,b.e11.B20TRC5CNBDRD.f09_g16.001,ocn,pop.h.ecosys.nyear1,DIA_IMPVF_Fe,1850-2005,1,/glade/campaign/cesm/collections/cesmLE/CESM-C...,402,CTRL,1
1,20C,b.e11.B20TRC5CNBDRD.f09_g16.002,ocn,pop.h.ecosys.nyear1,DIA_IMPVF_Fe,1920-2005,2,/glade/campaign/cesm/collections/cesmLE/CESM-C...,1920,20C,1
2,20C,b.e11.B20TRC5CNBDRD.f09_g16.009,ocn,pop.h.ecosys.nyear1,DIA_IMPVF_Fe,1920-2005,9,/glade/campaign/cesm/collections/cesmLE/CESM-C...,1920,20C,1
3,20C,b.e11.B20TRC5CNBDRD.f09_g16.010,ocn,pop.h.ecosys.nyear1,DIA_IMPVF_Fe,1920-2005,10,/glade/campaign/cesm/collections/cesmLE/CESM-C...,1920,20C,1
4,20C,b.e11.B20TRC5CNBDRD.f09_g16.011,ocn,pop.h.ecosys.nyear1,DIA_IMPVF_Fe,1920-2005,11,/glade/campaign/cesm/collections/cesmLE/CESM-C...,1920,20C,1


Now we can use the [`to_dataset_dict`](https://intake-esm.readthedocs.io/en/latest/api.html#intake_esm.core.esm_datastore.to_dataset_dict) method to return a dictionary of `xarray.Dataset`'s. `intake_esm` makes groups of these according to rules in the collection spec file.

We can use the `preprocess` parameter to pass in a function that makes some corrections to the dataset. So first we define a function that does the following:
- fix the time coordinate to be the middle of the interval
- drop the singleton dimension on SST (which screws up coordinate alignment)
- subset to the time-interval 1920-2100

In [49]:
client

Client Scheduler: tcp://10.12.206.28:39636 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/8787/status,Cluster Workers: 5 Cores: 10 Memory: 1.16 TiB


In [50]:
%%time
dsets = col_sub.to_dataset_dict(cdf_kwargs={'chunks': {'time':5}, 'decode_times': False})
#dsets


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component,experiment,stream'


CPU times: user 13.4 s, sys: 591 ms, total: 14 s
Wall time: 44.2 s


In [54]:
dsets['ocn,20C,pop.h.ecosys.nyear1'].HDIFB_Fe.attrs

{'long_name': 'Fe Horizontal Diffusive Flux across Bottom Face',
 'units': 'mmol/m^3/s',
 'grid_loc': '3113',
 'cell_methods': 'time: mean'}

In [55]:
dsets['ocn,20C,pop.h.ecosys.nyear1'].WT_Fe.attrs

{'long_name': 'Fe Flux Across Top Face',
 'units': 'mmol/m^3/s',
 'grid_loc': '3112',
 'cell_methods': 'time: mean'}

In [57]:
dsets['ocn,20C,pop.h.ecosys.nyear1'].DIA_IMPVF_Fe.attrs

{'long_name': 'Fe Flux Across Bottom Face from Diabatic Implicit Vertical Mixing',
 'units': 'mmol/m^3 cm/s',
 'grid_loc': '3113',
 'cell_methods': 'time: mean'}

In [58]:
def fix_time(ds):
    ds = ds.copy(deep=True)
    
    time_attrs = ds.time.attrs
    time_encoding = ds.time.encoding
    
    ds['time'] = xr.DataArray(
        cftime.num2date(
            ds.time_bound.mean(dim='d2'), 
            units=ds.time.units, 
            calendar=ds.time.calendar
        ), 
        dims=('time')
    )
    
    time_encoding['units'] = time_attrs.pop('units')
    time_encoding['calendar'] = time_attrs.pop('calendar')
    
    ds.time.attrs = time_attrs
    ds.time.encoding = time_encoding
    return ds    



def derive_var_Fe_FLUX_IN_100m(ds):
    """compute Fe flux across 100m (positive down)"""
    k_100m_top = np.where(ds.z_w_top == 100e2)[0][0]
    k_100m_bot = np.where(ds.z_w_bot == 100e2)[0][0]
    DIA_IMPVF = (-1.0) * ds.DIA_IMPVF_Fe.isel(z_w_bot=k_100m_bot)
    HDIFB = (-1.0) * ds.HDIFB_Fe.isel(z_w_bot=k_100m_bot) * ds.dz[k_100m_bot]
    WT = ds.WT_Fe.isel(z_w_top=k_100m_top) * ds.dz[k_100m_top]

    ds['Fe_FLUX_IN_100m'] = (DIA_IMPVF + HDIFB + WT)
    ds.Fe_FLUX_IN_100m.attrs = ds.DIA_IMPVF_Fe.attrs
    ds.Fe_FLUX_IN_100m.attrs['long_name'] = 'Fe flux across 100 m (positive up)'
    ds.Fe_FLUX_IN_100m.encoding = ds.WT_Fe.encoding
    return ds.drop(['DIA_IMPVF_Fe', 'HDIFB_Fe', 'WT_Fe'])

def compute_Fe_KPP_zint(ds):
    """compute KPP zint """
    
    dz100m = ds.dz.isel(z_t=slice(0, 10))
    ds['Fe_KPP_zint'] = (ds.KPP_SRC_Fe.isel(z_t=slice(0, 10)) * dz100m).sum(dim='z_t')
    ds.Fe_KPP_zint.attrs = ds.KPP_SRC_Fe.attrs
    ds.Fe_KPP_zint.attrs['long_name'] = '100m integral of Fe tendency from KPP non local mixing term'
    ds.Fe_KPP_zint.attrs['units'] = ds.KPP_SRC_Fe.attrs['units'] + ' cm'

    
    return ds.drop(['KPP_SRC_Fe'])

In [59]:
%%time

# fix time
dsets2 = {key: fix_time(ds) for key, ds in dsets.items()}
print('fixed time')

# subset time
dsets2 = {key: ds.sel(time=slice('1920', '2100')) for key, ds in dsets2.items()}
print('subset time done')

# derive Fe flux up at 100m
dsets2 = {key: derive_var_Fe_FLUX_IN_100m(ds) for key, ds in dsets2.items()}

# derive Fe flux up at 100m
dsets2 = {key: compute_Fe_KPP_zint(ds) for key, ds in dsets2.items()}


fixed time
subset time done
CPU times: user 342 ms, sys: 19.4 ms, total: 361 ms
Wall time: 935 ms


Concatenate the datasets in time, i.e. 20C + RCP8.5 experiments.

In [60]:
ordered_dsets_keys = ['ocn,20C,pop.h.ecosys.nyear1', 'ocn,RCP85,pop.h.ecosys.nyear1']
#ordered_dsets_keys = ['ocn.20C.pop.h', 'ocn.RCP85.pop.h']
ds = xr.concat(
    [dsets2[exp] for exp in ordered_dsets_keys], 
    dim='time', 
    data_vars='minimal',
    #compat='override' ## added this
)
#time_encoding = dsets2[ordered_dsets_keys[0]].time.encoding
#ds

In [61]:
variables= ['Fe_FLUX_IN_100m','Fe_KPP_zint']

In [62]:
# ds['time']=np.arange(1920,2101,1)

In [63]:
for var in variables:
    ds[var] = ds[var].chunk((5,34,384,320))

In [64]:
%%time
ds.load()

CPU times: user 1min 26s, sys: 14.9 s, total: 1min 41s
Wall time: 12min 54s


<xarray.Dataset>
Dimensions:             (d2: 2, member_id: 34, nlat: 384, nlon: 320, time: 181, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * time                (time) object 1920-07-02 12:00:00 ... 2100-07-02 12:0...
    TLAT                (nlat, nlon) float64 -79.22 -79.22 ... 72.19 72.19
    TLONG               (nlat, nlon) float64 320.6 321.7 322.8 ... 319.4 319.8
    ULAT                (nlat, nlon) float64 -78.95 -78.95 ... 72.41 72.41
    ULONG               (nlat, nlon) float64 321.1 322.3 323.4 ... 319.6 320.0
  * z_t                 (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m            (z_t_150m) float32 500.0 1.5e+03 ... 1.35e+04 1.45e+04
  * z_w                 (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot             (z_w_bot) float32 1e+03 2e+03 3e+03 ... 5.25e+05 5.5e+05
  * z_w_top             (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * member_id           (member_id) int64 1 2 9 10 11 12 ... 101 102 103 104 105
Dimensions without coordinates: d2, nlat, nlon
Data variables: (12/52)
    ANGLE               (nlat, nlon) float64 0.0 0.0 0.0 ... -0.0259 2.804e-07
    ANGLET              (nlat, nlon) float64 0.0 0.0 0.0 ... -0.03805 -0.01268
    DXT                 (nlat, nlon) float64 1.894e+06 1.893e+06 ... 1.473e+06
    DXU                 (nlat, nlon) float64 2.397e+06 2.397e+06 ... 1.391e+06
    DYT                 (nlat, nlon) float64 5.94e+06 5.94e+06 ... 5.046e+06
    DYU                 (nlat, nlon) float64 5.94e+06 5.94e+06 ... 5.493e+06
    ...                  ...
    sound               float64 1.5e+05
    stefan_boltzmann    float64 5.67e-08
    time_bound          (time, d2) float64 7.008e+05 7.012e+05 ... 7.669e+05
    vonkar              float64 0.4
    Fe_FLUX_IN_100m     (member_id, time, nlat, nlon) float32 nan nan ... nan
    Fe_KPP_zint         (member_id, time, nlat, nlon) float32 0.0 0.0 ... 0.0
Attributes: (12/13)
    Conventions:               CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netc...
    contents:                  Diagnostic and Prognostic Variables
    calendar:                  All years have exactly  365 days.
    intake_esm_varname:        DIA_IMPVF_Fe\nHDIFB_Fe\nKPP_SRC_Fe\nWT_Fe
    nco_openmp_thread_number:  1
    nsteps_total:              9100
    ...                        ...
    history:                   Mon Sep  2 14:20:41 2013: /glade/apps/opt/nco/...
    source:                    CCSM POP2, the CCSM Ocean Component
    cell_methods:              cell_methods = time: mean ==> the variable val...
    revision:                  $Id: tavg.F90 41939 2012-11-14 16:37:23Z mlevy...
    tavg_sum:                  31449600.0
    intake_esm_dataset_key:    ocn,20C,pop.h.ecosys.nyear1

#### write out data ANNUAL

In [69]:
%%time

for var in variables:

    print('starting variable: ', var)
 
    keep_vars = ['TAREA','time','dz','KMT', 'member_id','TLAT','TLONG', var] #'time_bound',

    ds_out = ds.drop([v for v in ds.variables if v not in keep_vars])

    outfile='/glade/scratch/kristenk/CESM-LE-output/CESM-LE-'+var+'.nc'
    ds_out.to_netcdf(outfile)

starting variable:  Fe_FLUX_IN_100m
starting variable:  Fe_KPP_zint
CPU times: user 56.7 s, sys: 2.01 s, total: 58.7 s
Wall time: 59.1 s


In [66]:
cluster.close()

In [67]:
#ds_out.time